In [823]:
import pandas as pd
import numpy as np

In [824]:
df=pd.read_csv('hospitalizacion_modelado.csv')
df.head()

# voy a observar que sucede si no tomo dias complicacion (la definicion es algo ambigua y no se entiende de donde sale)
# ademas de que al parecer overfitea el modelo


,muestrasTomadas,diabetes,hospitalizacionUltimoMes,antibioticoProfilaxis_OROQUINOLONAS,itu,tipoCultivo_HEMOCULTIVO,tipoCultivo_HEMOCULTIVO Y UROCULTIVO,tipoCultivo_UROCULTIVO,agenteAislado_E.COLI,agenteAislado_PSEUDOMONAS AERUGINOSA,"patronResistencia_ AMPI R, CIPRO R, GENTA R, SULFA M R",patronResistencia_MULTI SENSIBLE,"patronResistencia_RESISTENTE A AMPI, CIPRO Y GENTA","patronResistencia_RESISTENTE A AMPI, SULFA, CEFADROXILO, CEFUROXIMO, CIPRO Y CEFEPIME, CEFOTAXIMA",hospitalizacion,psa,antibioticoProfilaxis_OTROS,tipoCultivo_NO,agenteAislado_NO,patronResistencia_NO
0,12.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.386294,0,1,1,1
1,12.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2.041220,0,1,1,1
2,24.0,0,0,0,0,1,0,0,1,0,1,0,0,0,1,3.367296,0,0,0,0
3,12.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1.945910,0,1,1,1
4,24.0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,3.367296,0,0,0,1


In [825]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# voy a escalar estas variables para evitar sesgos en el modelo
df['muestrasTomadas'] = scaler.fit_transform(np.reshape(df['muestrasTomadas'],(-1,1))).ravel()
df['psa'] = scaler.fit_transform(np.reshape(df['psa'],(-1,1))).ravel()


In [826]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 20 columns):
 #   Column                                                                                             Non-Null Count  Dtype  
---  ------                                                                                             --------------  -----  
 0   muestrasTomadas                                                                                    523 non-null    float64
 1   diabetes                                                                                           523 non-null    int64  
 2   hospitalizacionUltimoMes                                                                           523 non-null    int64  
 3   antibioticoProfilaxis_OROQUINOLONAS                                                                523 non-null    int64  
 4   itu                                                                                                523 non-null    int64  

In [827]:
x=[]
for i in df:
    if (i!='hospitalizacion'):
        x.append(np.array(df[i]).reshape(-1,1))

X=np.hstack(x)
y=np.array(df.hospitalizacion).reshape(-1,1)

In [828]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(sampling_strategy='not majority',random_state=42)

X , y = ros.fit_resample(X,y)

# verificar el oversampling
print('value_counts y:',np.unique(y,return_counts=True))
print('len X:',len(X))

value_counts y: (array([0, 1]), array([500, 500]))
len X: 1000


In [829]:
'''from sklearn.decomposition import TruncatedSVD
svd=TruncatedSVD(n_components=15,n_iter=2,random_state=42)
X=svd.fit_transform(X)'''

'from sklearn.decomposition import TruncatedSVD\nsvd=TruncatedSVD(n_components=15,n_iter=2,random_state=42)\nX=svd.fit_transform(X)'

In [830]:
from sklearn.model_selection import train_test_split

X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, y, test_size=0.25, random_state=42)

In [831]:
# primero voy a probar un DecisionTreeClassifier aislado
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,r2_score,classification_report

arbol = DecisionTreeClassifier(max_depth=6)

arbol.fit(X_Train,Y_Train)

Y_Pred_Arbol = arbol.predict(X_Train)

Y_Test_Pred_Arbol = arbol.predict(X_Test)

print('accuracy_score con datos de entrenamiento',accuracy_score(Y_Train,Y_Pred_Arbol))
print('accuracy score con datos de entrenamiento',accuracy_score(Y_Test,Y_Test_Pred_Arbol))

accuracy_score con datos de entrenamiento 0.8773333333333333
accuracy score con datos de entrenamiento 0.852


In [832]:
# el modelo presenta overfitting (es "demasiado" bueno prediciendo verdaderos negativos)
print(classification_report(Y_Train,Y_Pred_Arbol))

              precision    recall  f1-score   support

           0       1.00      0.76      0.86       377
           1       0.80      1.00      0.89       373

    accuracy                           0.88       750
   macro avg       0.90      0.88      0.88       750
weighted avg       0.90      0.88      0.88       750



In [833]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

estimator_list = [('arbol_decision',DecisionTreeClassifier(max_depth=3)),
                  ('vecinos',KNeighborsClassifier(n_neighbors=4)),
                  ('SVC',SVC(C=1))
                  ]

stack_model = StackingClassifier(estimator_list,final_estimator=LogisticRegression())

stack_model.fit(X_Train,Y_Train)

Y_Pred = stack_model.predict(X_Train)

Y_Test_Pred = stack_model.predict(X_Test)

In [834]:
print('accuracy score con datos de entrenamiento',accuracy_score(Y_Train,Y_Pred))
print('accuracy score con datos de prueba',accuracy_score(Y_Test,Y_Test_Pred))
print('r2 score con datos de entrenamiento',r2_score(Y_Train,Y_Pred))
print('r2 score con datos de prueba',r2_score(Y_Test,Y_Test_Pred))



accuracy score con datos de entrenamiento 0.928
accuracy score con datos de prueba 0.868
r2 score con datos de entrenamiento 0.7119918077669765
r2 score con datos de prueba 0.4718647973881315


In [835]:
# se puede ver que el stacking ha disminuido sensiblemente el overfitting del modelo, a la vez que ha mejorado su precision
print(classification_report(Y_Train,Y_Pred))


              precision    recall  f1-score   support

           0       0.92      0.93      0.93       377
           1       0.93      0.92      0.93       373

    accuracy                           0.93       750
   macro avg       0.93      0.93      0.93       750
weighted avg       0.93      0.93      0.93       750



In [836]:
# creo un pipeline
from sklearn.pipeline import Pipeline

pipe_stack = Pipeline([('stack',stack_model)])

In [837]:
import joblib

joblib.dump(pipe_stack, 'pipe_stack.pkl', compress=1)

['pipe_stack.pkl']

In [838]:
X[1]

array([-0.2794328 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.19914784,
        0.        ,  1.        ,  1.        ,  1.        ])

In [839]:
modelo = joblib.load('pipe_stack.pkl')

In [840]:
modelo.score(X_Test,Y_Test)

0.868

In [841]:
X[37]

array([-0.2794328 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.56999262,
        0.        ,  1.        ,  1.        ,  1.        ])

In [842]:
# veamos si el modelo esta haciendo buenas predicciones
modelo.predict([[-0.2794328 ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.56999262,
        0.        ,  1.        ,  1.        ,  1.         ]])

array([0])

In [843]:
# al parecer si, ya que me devuelve el valor esperado en y
y[37]

0